In [ ]:
import os
import pandas as pd 
import matplotlib.pyplot as plt 
import torch
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np


from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

import pandas as pd
labels = pd.read_csv('../input/head-ct-hemorrhage/labels.csv')
labels.head()

labels['id'] = str('%03d' % (labels['id'][0]))+".png"
label_df = labels[['id',' hemorrhage']]
classes=list(label_df[' hemorrhage'].unique())
from sklearn.model_selection import train_test_split
train_label_df, test_label_df = train_test_split(label_df, test_size=0.10,shuffle=True)
train_label_df.to_csv ('./train_csv.csv', index = False, header=True)
test_label_df.to_csv ('./test_csv.csv', index = False, header=True)
from torch.utils.data import Dataset
import pandas as pd
import os
from PIL import Image
import torch

class CTDataset(Dataset):
    def __init__(self, root_dir, annotation_file, transform=None):
        self.root_dir = root_dir
        self.annotations = pd.read_csv(annotation_file)
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_id = self.annotations.iloc[index, 0]
        img = Image.open(self.root_dir+ img_id).convert("L")
        y_label = torch.tensor(float(self.annotations.iloc[index, 1]))

        if self.transform is not None:
            img = self.transform(img)

        return (img, y_label)
transform = transforms.Compose(
    [transforms.Resize((1000,1000)),transforms.ToTensor(),
     transforms.Normalize((0.5), (0.5))])

batch_size = 1

trainset = CTDataset(root_dir='../input/head-ct-hemorrhage/head_ct/head_ct/', annotation_file='./train_csv.csv', transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = CTDataset(root_dir='../input/head-ct-hemorrhage/head_ct/head_ct/', annotation_file='./test_csv.csv', transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
class MyConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, dilation=1, padding=0, stride=1):
        super(MyConv2d, self).__init__()

        self.kernel_size = (kernel_size, kernel_size)
        self.kernel_size_number = kernel_size * kernel_size
        self.out_channels = out_channels
        self.dilation = (dilation, dilation)
        self.padding = (padding, padding)
        self.stride = (stride, stride)
        self.in_channels = in_channels
        self.weights = nn.Parameter(torch.randn(self.out_channels, self.in_channels, self.kernel_size_number))

    def forward(self, x):
        width = ((x.shape[2] + 2 * self.padding[0] - self.dilation[0] * (self.kernel_size[0] - 1) - 1)// self.stride[0]
        ) + 1
        height = ((x.shape[3] + 2 * self.padding[1] - self.dilation[1] * (self.kernel_size[1] - 1) - 1)// self.stride[1]
        ) + 1
        windows = F.unfold(
            x, kernel_size=self.kernel_size, padding=self.padding, dilation=self.dilation, stride=self.stride
        )

        windows = windows.transpose(1, 2).contiguous().view(-1, x.shape[1], self.kernel_size_number)
        windows = windows.transpose(0, 1)
        
        
        result = torch.zeros(
            [x.shape[0] * self.out_channels, width, height], dtype=torch.float32, device="cpu"
        )

        for channel in range(x.shape[1]):
            for i_convNumber in range(self.out_channels):
                xx = torch.matmul(windows[channel], self.weights[i_convNumber][channel]) 
                xx = xx.view(-1, width, height)
                result[i_convNumber * xx.shape[0] : (i_convNumber + 1) * xx.shape[0]] += xx
                
        result = result.view(x.shape[0], self.out_channels, width, height)
        return result  

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 =MyConv2d(in_channels=3, out_channels=1, kernel_size=6,dilation=2,stride=5,padding=1)
        #self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(39601, 2)
        

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.fc1(x)
        return x


net = Net()
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

In [ ]:
losses=[]
for epoch in range(6):  
    running_loss = 0.0
    i=0
    for inputs, labels in trainloader:
        optimizer.zero_grad()
        outputs = net(inputs.float())
        loss = criterion(outputs,labels.long())
        loss.backward()
        optimizer.step()        
        running_loss += loss.item()
        i+=1
        if i % 200 == 0:    
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0
    with torch.no_grad():
        val_loss=0
        for val_inputs,val_labels in testloader:
            val_outputs = net(val_inputs.float())
            loss = criterion(val_outputs,val_labels.long())     
            val_loss += loss.item()
        losses.append(val_loss)
        print("Validation loss:",val_loss)

print('Finished Training')